# Launch SFT jobs in the Notebook.
<a target="_blank" href="https://colab.research.google.com/github/ai-hero/llm-research-orchestration/blob/main/notebooks/fine_tuning_research.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
# Set all important env variables needed for the application to work

## NOTE: It's best practice don't set the here, set them in your secrets.

## wandb
%env WANDB_API_KEY=
%env WANDB_USERNAME=

## huggingface
%env HF_TOKEN=

# S3 endpoint to access data and save model
%env S3_ENDPOINT=
%env S3_ACCESS_KEY_ID=
%env S3_SECRET_ACCESS_KEY=
%env S3_REGION=
%env S3_SECURE=true

# Run custom code !!!! Security risk
%env ALLOW_CUSTOM_TESTS=true
%env ALLOW_CUSTOM_METRICS=true

In [ ]:
from aihero.research.finetuning.sft import sft

In [ ]:
import yaml

configs = """
project:
  name: "mmlu"

training:
  dataset:
    type: "s3"
    name: "fine-tuning-research/mmlu"
    format: "completion"
  model:
    base:
      type: "hf"
      name: "meta-llama/Llama-2-7b-hf"
    output:
      type: "hf"
      name: "rparundekar/llama2-7b-mmlu"
  trainer:
    packing: False
    max_seq_length: 512
  sft:
    per_device_train_batch_size: 24
    per_device_eval_batch_size: 24
    learning_rate: 0.0002
    lr_scheduler_type: "cosine"
    warmup_ratio: 0.1
    max_steps: 500
    gradient_accumulation_steps: 4
    gradient_checkpointing: True
    gradient_checkpointing_kwargs:
      use_reentrant: False
    logging_strategy: "steps"
    logging_steps: 5
    evaluation_strategy: "steps"
    eval_steps: 100
  peft:
    r: 64  # the rank of the LoRA matrices
    lora_alpha: 16 # the weight
    lora_dropout: 0.1 # dropout to add to the LoRA layers
    bias: "none" # add bias to the nn.Linear layers?
    task_type: "CAUSAL_LM"
    target_modules:  # the name of the layers to add LoRA
      - "q_proj"
      - "k_proj"
      - "v_proj"
      - "o_proj"
      - "gate_proj"
      - "up_proj"
      - "down_proj"
      -  "lm_head"
    quantized: True
"""

config = yaml.safe_load(configs)

In [ ]:
sft(config)